In [23]:
# paths 

# secrets file
secrets_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\user_files\secrets.json"
persistent_storage_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\persistent_storage_index"
docs_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs"

PERSIST_DIR = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\persistent_storage"
vector_index_dir = PERSIST_DIR + r"\vector"
summary_index_dir = PERSIST_DIR + r"\summary"


In [24]:
import json
import os
import torch
from llama_index.core import Settings, VectorStoreIndex, SummaryIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.openai import OpenAIEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader, PromptTemplate





# Check if CUDA is available
if torch.cuda.is_available():
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Using GPU: {gpu_name}")
else:
    print("CUDA is not available. Using CPU.")


with open(secrets_path, 'r') as file:
    data = json.load(file)
    

OPENAI_API_KEY = data["OPENAI_KEY"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Using GPU: NVIDIA GeForce GTX 1650


In [25]:
def check_persist(storage_path):
    return os.path.exists(storage_path)

In [26]:
gpt3_5_llm = OpenAI(model="gpt-3.5-turbo")
gpt3_5_embed = OpenAIEmbedding(model="text-embedding-ada-002")
# System prompt for the LLM




system_prompt = """
answer questions about a candidate's resume and profile
"""
phi3_llm = Ollama(
    model="phi3:3.8b-mini-4k-instruct-q4_K_M",
    temperature=0.01,
    request_timeout=400,
    system_prompt=system_prompt,
    context_window=2000
)

phi3_embed = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)


In [27]:
qa_prompt_tmpl_str_gpt4 = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge,
it is very important that you follow the instructions clearly and answer based on the given information.
Answer the query in the format requested in the query.
Do not leave blanks in the answers.
Always answer the questions in the form of a cover letter and in first person.
If asked for a cover letter, write a short cover letter talking about your previous work experience and how it would make you a good fit for the given role.
If asked about why you want to work at a certain company, write a concise cover letter including the company's name and talking about your previous work experience and how it would make you a good fit for the given role.

Query: {query_str}
Answer: \
"""

qa_prompt_tmpl_str_phi3 = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge,
it is very important that you follow the instructions clearly and answer based on the given information.
Answer the query in the format requested in the query.
If there are options in the query, answer by choosing one or more options as required.
Try to read the document thoroughly to extract more info from the document.
When asked for city, return the city name along with the state.
Return only one answer, do not return multiple answers or list of answers unless specified.
For queries which ask for years of experience, always return values greater than 1.
For queries like "how many years of experience do you have with some tool", return just the integer.
For queries like "how many years of experience", the answer should always be an integer.
For questions that start like "do you have experience with", always return "Yes".
For queries that begin with "Experience with", they are asking the number of years of experience; treat this query the same as those that ask for the number of years of experience with a certain tool.
For queries that ask "are you willing to relocate" or "are you local to a certain place", always answer "Yes".
Keep the answers concise and to the point, do not answer long sentences unless necessary or specified.
Keep the answers concise. Answer in one or two words wherever possible. Keep the answers short, do not elaborate unless necessary, do not explain or elaborate.

Query: {query_str}
Answer: \
"""

In [35]:




import json
import os
import torch
from llama_index.core import Settings, VectorStoreIndex, SummaryIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.openai import OpenAIEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader, PromptTemplate





# Check if CUDA is available
if torch.cuda.is_available():
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Using GPU: {gpu_name}")
else:
    print("CUDA is not available. Using CPU.")


with open(secrets_path, 'r') as file:
    data = json.load(file)
    

OPENAI_API_KEY = data["OPENAI_KEY"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


def check_persist(storage_path):
    return os.path.exists(storage_path)



gpt3_5_llm = OpenAI(model="gpt-3.5-turbo")
gpt3_5_embed = OpenAIEmbedding(model="text-embedding-ada-002")
# System prompt for the LLM




system_prompt = """
answer questions about a candidate's resume and profile
"""
phi3_llm = Ollama(
    model="phi3:3.8b-mini-4k-instruct-q4_K_M",
    temperature=0.01,
    request_timeout=400,
    system_prompt=system_prompt,
    context_window=2000
)

phi3_embed = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)




qa_prompt_tmpl_str_gpt4 = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge,
it is very important that you follow the instructions clearly and answer based on the given information.
Answer the query in the format requested in the query.
Do not leave blanks in the answers.
Always answer the questions in the form of a cover letter and in first person.
If asked for a cover letter, write a short cover letter talking about your previous work experience and how it would make you a good fit for the given role.
If asked about why you want to work at a certain company, write a concise cover letter including the company's name and talking about your previous work experience and how it would make you a good fit for the given role.

Query: {query_str}
Answer: \
"""

qa_prompt_tmpl_str_phi3 = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge,
it is very important that you follow the instructions clearly and answer based on the given information.
Answer the query in the format requested in the query.
If there are options in the query, answer by choosing one or more options as required.
Try to read the document thoroughly to extract more info from the document.
When asked for city, return the city name along with the state.
Return only one answer, do not return multiple answers or list of answers unless specified.
For queries which ask for years of experience, always return values greater than 1.
For queries like "how many years of experience do you have with some tool", return just the integer.
For queries like "how many years of experience", the answer should always be an integer.
For questions that start like "do you have experience with", always return "Yes".
For queries that begin with "Experience with", they are asking the number of years of experience; treat this query the same as those that ask for the number of years of experience with a certain tool.
For queries that ask "are you willing to relocate" or "are you local to a certain place", always answer "Yes".
Keep the answers concise and to the point, do not answer long sentences unless necessary or specified.
Keep the answers concise. Answer in one or two words wherever possible. Keep the answers short, do not elaborate unless necessary, do not explain or elaborate.

Query: {query_str}
Answer: \
"""

# Function to get the summarization tool using GPT-3.5
def get_summary_tool(docs):
    # Split documents into chunks
    splitter = SentenceSplitter(chunk_size=1024)
    documents = splitter.get_nodes_from_documents(docs)


    # Set LLM and embedding model for the summarizer tool
    Settings.llm = gpt3_5_llm
    Settings.embed_model = gpt3_5_embed

    # Create SummaryIndex and QueryEngine for summarization
    # summary_index = SummaryIndex(documents)
    
    if check_persist(summary_index_dir):
        storage_context = StorageContext.from_defaults(persist_dir=summary_index_dir)
        summary_index = load_index_from_storage(storage_context)
    else:
        summary_index = VectorStoreIndex(documents)
        summary_index.storage_context.persist(summary_index_dir)
        

    summary_query_engine = summary_index.as_query_engine(
            response_mode="refine",
            use_async=True,
        )

    qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str_phi3)

    # Update query engine prompts
    summary_query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})



    # Create and return the summary tool
    summary_tool = QueryEngineTool.from_defaults(
        name="coverletter_tool",
        query_engine=summary_query_engine,
        description="for long answer and summary based questions about the profile"
    )

    return summary_tool




# Function to get the vector search tool using phi3-mini
def get_vector_tool(docs):
    # Split documents into chunks
    splitter = SentenceSplitter(chunk_size=1024)
    documents = splitter.get_nodes_from_documents(docs)

    # Set LLM and embedding model for the vector search tool
    Settings.llm = phi3_llm
    Settings.embed_model = phi3_embed

    # Create VectorStoreIndex and QueryEngine for vector search
    
    if check_persist(vector_index_dir):
        storage_context = StorageContext.from_defaults(persist_dir=vector_index_dir)
        vector_index = load_index_from_storage(storage_context)
    else:
        vector_index = VectorStoreIndex(documents)
        vector_index.storage_context.persist(vector_index_dir)

    vector_query_engine = vector_index.as_query_engine(response_mode = 'compact' , use_async = True)
        
    qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str_phi3)

    # Update query engine prompts
    vector_query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})

    # Create and return the vector tool
    vector_tool = QueryEngineTool.from_defaults(
        name="vector_tool",
        query_engine=vector_query_engine,
        description="Useful for retrieving specific context from the documents."
    )

    return vector_tool

# Example usage:
# Assuming `docs` is the data you want to process
docs = SimpleDirectoryReader(docs_path).load_data()

# Get the summarization and vector tools
summary_tool = get_summary_tool(docs)
vector_tool = get_vector_tool(docs)

# Now you can use summary_tool and vector_tool for your queries


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)



query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    
)


response = query_engine.query("how many years of experience does he have in python")
print(str(response.response))


Using GPU: NVIDIA GeForce GTX 1650
5


In [29]:



query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    
)


response = query_engine.query("how many years of experience does he have in python")
print(str(response.response))

5


In [30]:
response = query_engine.query("when did you graduate")
print(str(response.response))

May 2024
